In [ ]:
segmentation_path = '/group/jug/ashesh/data/Stefania/20230327_Ki67_and_Iba1_trainingdata/Iba1/label_iba1_nucleus.tif'
image_path = '/group/jug/ashesh/data/Stefania/20230327_Ki67_and_Iba1_trainingdata/Iba1/iba1_nucleus.tif'
test_idx = 1 # obtained from looking into the train_val_data

In [ ]:
from ml_collections.config_dict import ConfigDict
from disentangle.core.data_split_type import DataSplitType
from disentangle.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType, get_train_val_data

data_config = ConfigDict()
data_config.subdset_type = SubDsetType.OnlyIba1
datadir = '/group/jug/ashesh/data/Stefania/20230327_Ki67_and_Iba1_trainingdata/'
data = get_train_val_data(datadir, data_config, DataSplitType.All, val_fraction=0.1, test_fraction=0.1)


In [ ]:
from disentangle.core.tiff_reader import load_tiff
data_v2 = load_tiff(image_path)
seg_v2 = load_tiff(segmentation_path)

In [ ]:
seg_v2.shape

## Dilation operation

In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import binary_dilation
import numpy as np
   
def gkern(l=5, sig=1.):
    """\
    creates gaussian kernel with side length `l` and a sigma of `sig`
    """
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

import numpy as np
import cv2

def dilate_instance_segmentation(mask, kernel):
    """
    Dilate an instance segmentation mask.
    
    Parameters:
    - mask: numpy array of shape (H, W), where each unique value represents a different instance.
    - kernel: numpy array for the dilation operation (e.g., cv2.getStructuringElement).

    Returns:
    - dilated_mask: numpy array of shape (H, W), with the same unique values dilated.
    """
    # Initialize the dilated mask with zeros
    dilated_mask = np.zeros_like(mask)
    
    # Get unique instance IDs in the mask (excluding background, assumed to be 0)
    unique_instances = np.unique(mask)
    if unique_instances[0] == 0:
        unique_instances = unique_instances[1:]  # remove background if present

    # Process each instance separately
    for instance_id in unique_instances:
        # Create a boolean mask for the current instance
        instance_mask = (mask == instance_id).astype(np.uint8)
        
        # Dilate the current instance mask
        dilated_instance = binary_dilation(instance_mask, structure=kernel)
        
        # Update the dilated mask with the dilated instance
        dilated_mask[dilated_instance > 0] = instance_id

    return dilated_mask


img = seg_v2[0,:500,:500].copy()
# plt.imshow()

In [ ]:
kernel = gkern(l =20, sig=1 ) > 1e-4
new_img = dilate_instance_segmentation(img, kernel )
_,ax = plt.subplots(1,3, figsize=(15,5))
ax[0].imshow(img[30:100,120:200])
ax[1].imshow(new_img[30:100,120:200])
ax[2].imshow(new_img[30:100,120:200] - img[30:100,120:200])

In [ ]:
data_v2.shape

In [ ]:
fnames = ['Iba1only_1.czi',
'Iba1only_11.czi',
'Iba1only_13.czi',
'Iba1only_15.czi',
'Iba1only_3.czi',
'Iba1only_5.czi',
'Iba1only_7.czi',
'Iba1only_9.czi',
'Iba1only_10.czi',
'Iba1only_12.czi',
'Iba1only_14.czi',
'Iba1only_2.czi',
'Iba1only_4.czi',
'Iba1only_6.czi',
'Iba1only_8.czi']
fnames_v2_order = sorted(fnames)
print(fnames_v2_order)
new_indices = [int(fname.replace('.czi','').split('_')[-1])-1 for fname in fnames_v2_order]
a,b = zip(*sorted(zip(new_indices, range(len(new_indices))), key=lambda x: x[0]))
data_v3 = data_v2[b,:,:]
seg_v3 = seg_v2[b,:,:]

In [ ]:
print(a,b)

In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(10,5),ncols=2)
idx = 8
ax[0].imshow(seg_v3[idx])
ax[1].imshow(data[idx,...,0])

In [ ]:
test_data = get_train_val_data(datadir, data_config, DataSplitType.Test, val_fraction=0.1, test_fraction=0.1)


In [ ]:
test_img = test_data[0,...,0].copy()
raw_seg_mask = seg_v3[1].copy()
# seg_mask = dilate_instance_segmentation(raw_seg_mask, kernel)
seg_mask = raw_seg_mask
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(15,5),ncols=3)
ax[0].imshow(raw_seg_mask)
ax[1].imshow(seg_mask)
ax[2].imshow(test_img)

In [ ]:
break here

In [ ]:
import numpy as np
empty_patches = [
    (0,600,250,650),
    (100,440,1600,2160),
    (300,750,1900,2464),
    (500,700,1000,1800)
]
for hs,he,ws,we in empty_patches:
    assert np.unique(seg_mask[hs:he,ws:we]) == 0


In [ ]:
instance_idx = np.unique(seg_mask)
instance_idx = instance_idx[instance_idx!=0]
print(len(instance_idx))
print(instance_idx)

In [ ]:
def get_empty_patch_coordinates(hN, wN):
    patch_coord = None
    for idx in np.random.permutation(np.arange(len(empty_patches))):
        patch_coord = empty_patches[idx]
        if patch_coord[1] - patch_coord[0] > hN and patch_coord[3] - patch_coord[2] > wN:
            break
    return patch_coord


def replace_one_nuclues(instance_id, seg_mask, img, extra_margin=0):
    one_ins = seg_mask==instance_id
    x,y = np.where(one_ins)
    hs, he, ws, we = x.min(), x.max(), y.min(), y.max()
    if extra_margin:
        hs = max(0, hs-extra_margin)
        he = min(seg_mask.shape[0], he+extra_margin)
        ws = max(0, ws-extra_margin)
        we = min(seg_mask.shape[1], we+extra_margin)
    hN = he-hs
    wN = we-ws
    patch_coord = get_empty_patch_coordinates(hN, wN)
    empty_hs, empth_ws = np.random.randint(patch_coord[0], patch_coord[1]-hN), np.random.randint(patch_coord[2], patch_coord[3]-wN)
    img[hs:he,ws:we] = img[empty_hs:empty_hs+hN, empth_ws:empth_ws+wN]
    return (hs,he,ws,we)

In [ ]:
test_img = test_data[0,...,0].copy()
percent = 0.7
extra_margin = 0
removed_ids = np.random.choice(instance_idx, int(len(instance_idx)*percent), replace=False)
for idx in removed_ids:
    replace_one_nuclues(idx, seg_mask, test_img, extra_margin=extra_margin)

_,ax = plt.subplots(figsize=(20,5),ncols=4)
ax[0].imshow(test_img)
ax[1].imshow(test_img[300:800,:500])
ax[2].imshow(test_img[1000:1500,:500])
ax[3].imshow(test_img[1000:1500,1300:1800])

In [ ]:
break here

In [ ]:
from disentangle.core.tiff_reader import save_tiff
import os
num_imgs = 10
for img_idx in range(num_imgs):
    test_img = test_data[0,...,0].copy()
    percent = 0.7
    extra_margin = 5
    removed_ids = np.random.choice(instance_idx, int(len(instance_idx)*percent), replace=False)
    for idx in removed_ids:
        replace_one_nuclues(idx, seg_mask, test_img, extra_margin=extra_margin)
    output_path = f'/group/jug/ashesh/data/Stefania/20230327_Ki67_and_Iba1_trainingdata/synthetic_test/percent{int(percent*100)}/iba1_nucleus_test_{img_idx}_margin{extra_margin}.tif'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    save_tiff(output_path, test_img)
    print('Saved: ', output_path)

In [ ]:
new_data = load_tiff('/group/jug/ashesh/data/Stefania/20230327_Ki67_and_Iba1_trainingdata/synthetic_test/percent70/iba1_nucleus_test_9_margin5.tif')
old_data = test_data[0,...,0].copy()

In [ ]:
import seaborn as sns
sns.histplot(new_data[::4,::4].ravel(), color='r', alpha=0.5,log_scale=True)
sns.histplot(old_data[::4,::4].ravel(), color='b', alpha=0.5,log_scale=True)

In [ ]:
print(np.quantile(new_data, [0.1,0.5,0.9, 0.99]), new_data.mean(), new_data.std())
print(np.quantile(old_data, [0.1,0.5,0.9, 0.99]), old_data.mean(), old_data.std())

In [ ]:
_,ax = plt.subplots(figsize=(10,5),ncols=2)
ax[0].imshow(test_img)
ax[1].imshow(test_data[0,...,0])